In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline

from skfeature.function.information_theoretical_based import MRMR


In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

# Read the data
df = pd.read_csv("diabetes_prediction_dataset.csv")

# Handle duplicates
df = df.drop_duplicates()

# Remove unnecessary values [0.00195%]
df = df[df['gender'] != 'Other']


In [3]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'

df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)


In [4]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df

data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')


In [5]:
# Feature Selection (MRMR)
X = data.drop('diabetes', axis=1)
y = data['diabetes']
feat_idx = MRMR.mrmr(X.values, y.values, n_selected_features=10)
selected_features = X.columns[feat_idx].tolist()
data = data[selected_features + ['diabetes']]  # Ensure diabetes column is included

# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)


In [6]:
# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), data.drop('diabetes', axis=1).select_dtypes(include='number').columns),
        ('cat', OneHotEncoder(), data.drop('diabetes', axis=1).select_dtypes(include='object').columns)
    ])

# Split data into features and target variable
X = data.drop('diabetes', axis=1)
y = data['diabetes']

In [7]:
# Define a pipeline that preprocesses the data, resamples data, and then trains a classifier
clf = imbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('over', over),
    ('under', under),
    ('classifier', RandomForestClassifier())
])

# Define the hyperparameters and the values you want to test
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

In [8]:
# Create Grid Search object
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Train the model
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['blood_glucose_level', 'HbA1c_level', 'heart_disease', 'age',
       'hypertension', 'bmi'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index([], dtype='object'))])),
                                       ('over', SMOTE(sampling_strategy=0.1)),
                                       ('under',
                                        RandomUnderSampler(sampling_strategy=0.5)),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [None, 10, 20],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 200]})

In [10]:
# Predict on the test set using the best model
y_pred = grid_search.predict(X_test)

# Evaluate the model
print("Model Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Model Accuracy:  0.9468948299178196
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     17525
           1       0.67      0.80      0.73      1701

    accuracy                           0.95     19226
   macro avg       0.82      0.88      0.85     19226
weighted avg       0.95      0.95      0.95     19226

